# Creating a Migration Algorithm

This tutorial demonstrates how we can create a simple migration algorithm on EdgeSimPy.

First, let's import the EdgeSimPy modules:

In [ ]:
try:
    # Importing EdgeSimPy components
    from edge_sim_py import *
    import networkx as nx
    import msgpack

    # Importing Matplotlib, Pandas, and NumPy for logs parsing and visualization
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np

except ModuleNotFoundError:
    # Downloading EdgeSimPy binaries from GitHub (the "-q" parameter suppresses Pip's output. You check the full logs by removing it)
    %pip install -q git+https://github.com/EdgeSimPy/EdgeSimPy.git@v1.1.0

    # Downloading Pandas, NumPy, and Matplotlib (these are not directly used here, but they can be useful for logs parsing and visualization)
    %pip install -q pandas==1.3.5
    %pip install -q numpy==1.26.4
    %pip install -q matplotlib==3.5.0

    # Importing EdgeSimPy components and its built-in libraries (NetworkX and MessagePack)
    from edge_sim_py import *
    import networkx as nx
    import msgpack

    # Importing Matplotlib, Pandas, and NumPy for logs parsing and visualization
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np

## Implementing the Migration Algorithm

Our simple migration algorithm will work according to the well-known Worst-Fit heuristic. In a nutshell, it will provision each service to the edge server with the largest amount of free resources.

In [ ]:
def my_algorithm(parameters):
    # Let's iterate over the list of services using the 'all()' helper method
    print("\n\n")
    print(f"==== TIME STEP {parameters['current_step']} ====")
    print("---- EDGE SERVERS ----")
    for server in EdgeServer.all():
        server_metadata = {
            "server": server,
            "capacity": [server.cpu, server.memory, server.disk],
            "demand": [server.cpu_demand, server.memory_demand, server.disk_demand],
            "container_layers": [layer.id for layer in server.container_layers],
            "services": [service for service in server.services],
        }
        print(server_metadata)

    print("")
    print("---- SERVICES ----")
    for service in Service.all():
        service_image = ContainerImage.find_by(attribute_name="digest", attribute_value=service.image_digest)
        service_layers = [ContainerLayer.find_by(attribute_name="digest", attribute_value=layer).id for layer in service_image.layers_digests]

        service_metadata = {
            "service": service,
            "requirements": [service.cpu_demand, service.memory_demand],
            "layers": service_layers,
            "server": service.server,
            "available": service._available,
            "being_provisioned": service.being_provisioned,
        }
        print(service_metadata)

    # We don't want to migrate services are are already being migrated. As we want to avoid excessive migrations, we are going to
    # define a cool down period for services that have been migrated. For simplicity, we are going to set the cool down period as
    # 10 time steps. To get such information, we are going to access the service's last migration time step and compare it with
    # the current time step. If the difference between the current time step and the last migration time step is less than 10,
    # we are going to skip the service.
    COOLDOWN_STEPS = 10

    for service in Service.all():
        # Gathering the current time step and the last migration time step of the service
        current_step = parameters["current_step"]
        
        if len(service._Service__migrations) > 0 and service._Service__migrations[-1]["end"] is not None:
            last_migration_ended_at = service._Service__migrations[-1]["end"]
        else:
            last_migration_ended_at = None

        # Checking if the service is not being provisioned and if the cool down period has being reached
        service_has_not_been_migrated_yet = len(service._Service__migrations) == 0
        service_not_being_provisioned = service.being_provisioned is False
        cooldown_has_been_reached = last_migration_ended_at is not None and current_step - last_migration_ended_at >= COOLDOWN_STEPS

        if service_has_not_been_migrated_yet or service_not_being_provisioned and cooldown_has_been_reached:
            # We need to sort edge servers based on amount of free resources they have. To do so, we are going to use Python's
            # "sorted" method (you can learn more about "sorted()" in this link: https://docs.python.org/3/howto/sorting.html).
            # The capacity of edge servers is modeled in three layers (CPU, memory, and disk). For simplicity, we are going to
            # use only the CPU layer to sort edge servers. We calculate the free CPU resources of each edge server by subtracting
            # the CPU demand of the services hosted in the edge server from the total CPU capacity of the edge server. We then
            # sort edge servers based on their free CPU resources. To do so, we use the "sorted" method and set the "key" attribute
            # as a lambda function that calculates the free CPU resources of edge servers. We set the "reverse" attribute as "True"
            # as we want to sort edge servers by their free CPU resources in descending order.
            edge_servers = sorted(
                EdgeServer.all(),
                key=lambda s: s.cpu - s.cpu_demand,
                reverse=True,
            )

            for edge_server in edge_servers:
                # Checking if the edge server has resources to host the service
                if edge_server.has_capacity_to_host(service=service):
                    # We just need to migrate the service if it's not already in the least occupied edge server
                    if service.server != edge_server:
                        print(f"\t\t[STEP {parameters['current_step']}] Migrating {service} From {service.server} to {edge_server}")

                        service.provision(target_server=edge_server)

                        # After start migrating the service we can move on to the next service
                        break

## Running the Simulation

Before testing our algorithm, we must tell EdgeSimPy when it must stop the simulation. For example, let's run the simulation for 600 seconds (i.e., 10 minutes).

To do so, we must create a simple function that will be used as the simulation's stopping criterion. Behind the scenes, at the end of each time step, EdgeSimPy will run that function, halting the simulation if it returns `True`.

In [ ]:
def stopping_criterion(model: object):    
    # As EdgeSimPy will halt the simulation whenever this function returns True,
    # its output will be a boolean expression that checks if the current time step is 600
    return model.schedule.steps == 600

Once we have developed our stopping criterion function, we can create an instance of the `Simulation` class passing a couple of arguments, load a dataset, and run the simulation to check how our algorithm goes.

In [ ]:
# Creating a Simulator object
simulator = Simulator(
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset from GitHub
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset1.json")

# Executing the simulation
simulator.run_model()